In [3]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [5]:
def simulate_next_conditions(
    internal_temp, external_temp, internal_humidity, external_humidity, heating, venting
):
    temp = internal_temp
    humidity = internal_humidity

    temp_rate = 0.15 if not (heating or venting) else 0.05
    humidity_rate = 0.10 if not (heating or venting) else 0.03

    temp += temp_rate * (external_temp - internal_temp)
    humidity += humidity_rate * (external_humidity - internal_humidity)

    if heating:
        temp += 2.0
        humidity_reduction = min(4.0, humidity * 0.3)
        humidity = max(0, humidity - humidity_reduction)

    if venting:
        temp += 0.4 * (external_temp - temp)
        humidity += 0.35 * (external_humidity - humidity)

    if not venting and humidity < 90:
        if humidity < 85:
            passive_gain = 0.08 * (85 - humidity)
        else:
            passive_gain = 0.02
        humidity += passive_gain

    humidity = max(0, min(100, humidity))

    return temp, humidity

In [6]:
n_samples = 1000
start_time = datetime.now()
datetimes = [start_time + timedelta(hours=1) for i in range(n_samples)]

external_temps = np.random.normal(loc=80, scale=40, size=n_samples)
external_humidies = np.random.uniform(50, 90, size=n_samples)

In [7]:
internal_temp = np.random.uniform(60, 80)
internal_humidity = np.random.uniform(60, 80)
data = []

for i in range(n_samples):
    ext_temp = external_temps[i]
    ext_hum = external_humidies[i]

    heating = int(internal_temp < 65)
    venting = int(internal_temp > 72)

    next_temp, next_hum = simulate_next_conditions(
        internal_temp, ext_temp, internal_humidity, ext_hum, heating, venting
    )

    data.append({
        "datetime": datetimes[i],
        "next_internal_temp": next_temp,
        "next_internal_hum": next_hum,
        "external_temp": ext_temp,
        "internal_temp": internal_temp,
        "external_humidity": ext_hum,
        "internal_humidity": internal_humidity,
        "heating": heating,
        "venting": venting,
    })

    internal_temp = next_temp
    internal_humidity = next_hum

In [5]:
df = pd.DataFrame(data)
df.to_csv("./data/synthetic_greenhouse_data.csv", index=False)

print("Generate synthetic data successfully.")
print(df.head())

Generate synthetic data successfully.
                    datetime  next_internal_temp  next_internal_hum  \
0 2025-07-14 01:02:22.513978           68.710835          69.538413   
1 2025-07-14 01:02:22.513978           66.513979          71.061493   
2 2025-07-14 01:02:22.513978           66.380080          72.508418   
3 2025-07-14 01:02:22.513978           67.933820          73.882997   
4 2025-07-14 01:02:22.513978           66.332029          75.188847   

   external_temp  internal_temp  external_humidity  internal_humidity  \
0      86.930467      67.751907          86.445592          67.935172   
1      24.773715      68.710835          76.323427          69.538413   
2      63.836000      66.513979          83.702181          71.061493   
3      97.454877      66.380080          63.699739          72.508418   
4      35.898007      67.933820          55.115025          73.882997   

   heating  venting  
0        0        0  
1        0        0  
2        0        0  
3       